In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import numpy as np

In [ ]:
df = load_dataset('imdb')

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
df.shape

{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}

In [ ]:
texts = df['train']['text'][:100]
labels = df['train']['label'][:100]

In [ ]:
len(texts), len(labels)

(100, 100)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size = 0.2, random_state = 42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size = 0.5, random_state = 42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)
test_encodings = tokenizer(test_texts, truncation = True, padding = True)

In [ ]:
train_df = tf.data.Dataset.from_tensor_slices(({key: train_encodings[key] for key in ['input_ids', 'attention_mask']}, train_labels)).shuffle(10).batch(1)
val_df = tf.data.Dataset.from_tensor_slices(({key: val_encodings[key] for key in ['input_ids', 'attention_mask']}, val_labels)).batch(1)
test_df = tf.data.Dataset.from_tensor_slices(({key: test_encodings[key] for key in ['input_ids', 'attention_mask']}, test_labels)).batch(1)

In [ ]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
loss_fn = SparseCategoricalCrossentropy(from_logits = True)

In [ ]:
bert_model.compile(loss = loss_fn, optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
history = bert_model.fit(train_df, validation_data = val_df, epochs = 3)

In [ ]:
test_loss, test_accuracy = bert_model.evaluate(test_df)
print('Test Loss: ', test_loss)
print('Test Accuracy: ', test_accuracy)